In [1]:
# import subprocess
import os


This notebook prepares the data for the rural beauty model, by reprojecting, cropping and or resampling of raw datasets.

To do so, it loads paths from the rural_beauty/rural_beauty/config.py. 

It then either calls python functions from the same module or calls bash and R scripts from the scripts folder. 
To do the latter, variable are passed to the OS. 

In [2]:
%load_ext rpy2.ipython

# Beauty/Uniqueness/Diversity

Transform the beauty shapefiles to raster data.


In [3]:
# import paths from config.py
from rural_beauty.config import bild_vector_dir, bild_raster_dir # load absolute paths as defined in config.py

# pass them to the OS
os.environ['bild_vector_dir'] = str(bild_vector_dir)
os.environ['bild_raster_dir'] = str(bild_raster_dir)


This is the rural_beauty module


Take every .shp file in the -i directory and rasterizes it into the -o direcory and adjusts the naming. 


In [4]:
!Rscript ../scripts/rasterize_ratings.R -i $bild_vector_dir -o $bild_raster_dir

Linking to GEOS 3.10.2, GDAL 3.4.1, PROJ 8.2.1; sf_use_s2() is TRUE
terra 1.7.83


# DEM

Take the digital elevation model and calculate the range of elevation in the 1x1 km grid. 

In [5]:
!pwd

/h/u145/hofer/MyDocuments/Granular/beauty/notebooks


In [2]:
# import paths from config.py
from rural_beauty.config import DEM_EU, DEM_EU_range, DEM_EU_range_scaled

# pass them to the OS
os.environ['DEM_EU'] = str(DEM_EU)
os.environ['DEM_range'] = str(DEM_EU_range)
os.environ['DEM_range_scaled'] = str(DEM_EU_range_scaled)

This is the rural_beauty module


In [9]:
!../scripts/process_dem_data_minmax_EU.sh $DEM_EU $DEM_range $DEM_range_scaled

/h/u145/hofer/MyDocuments/Granular/beauty/data/processed/dem/EU_DEM_EU_max.tif already exists. Skipping generation.
/h/u145/hofer/MyDocuments/Granular/beauty/data/processed/dem/EU_DEM_EU_min.tif already exists. Skipping generation.
/h/u145/hofer/MyDocuments/Granular/beauty/data/processed/dem/DEM_EU_range.tif already exists. Skipping generation.
Maximum Range: 1869.229
Generating scaled (0,1) output to /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/DEM_EU_range_scaled.tif
Input file size is 6566, 4060
0...10...20...30...40...50...60...70...80...90...100 - done.
Process completed. The final output is normalized between 0 and 1 at /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/DEM_EU_range_scaled.tif


# Process the OSM data. 

We downloaded the whole osm data from geofabrik on July 3rd from:

https://download.geofabrik.de/europe-latest.osm.pbf

https://download.geofabrik.de/germany-latest.osm.pbf


In [ ]:
# import paths from config.py
from rural_beauty.config import OSM_full_EU
from rural_beauty.config import powerlines_EU_vector, powerlines_EU_raster, powerlines_EU_raster_scaled
from rural_beauty.config import streets_EU_vector, streets_EU_raster, streets_EU_raster_scaled
from rural_beauty.config import windpower_EU_vector, windpower_EU_raster, windpower_EU_raster_scaled

import os
# pass them to the OS
os.environ['powerlines_vector'] = str(powerlines_EU_vector)
os.environ['powerlines_raster'] = str(powerlines_EU_raster)
os.environ['powerlines_raster_scaled'] = str(powerlines_EU_raster_scaled)

os.environ['streets_vector'] = str(streets_EU_vector)
os.environ['streets_raster'] = str(streets_EU_raster)
os.environ['streets_raster_scaled'] = str(streets_EU_raster_scaled)

os.environ['windpower_vector'] = str(windpower_EU_vector)
os.environ['windpower_raster'] = str(windpower_EU_raster)
os.environ['windpower_raster_scaled'] = str(windpower_EU_raster_scaled)

Subset the big .pbf file to windpower, streets, and powerlines

In [ ]:
!../scripts/osm_extract_streets.sh $OSM_full_EU $streets_vector
!../scripts/osm_extract_powerlines.sh $OSM_full_EU $powerlines_vector 
!../scripts/osm_extract_windpower.sh $OSM_full_EU $windpower_vector $windpower_EU_raster_scaled

In [ ]:
!../scripts/rasterize_OSM_line_geom_EU.R $powerlines_vector $powerlines_raster $powerlines_EU_raster_scaled
!../scripts/rasterize_OSM_line_geom_EU.R $streets_vector $streets_vector $streets_raster_scaled
!../scripts/rasterize_OSM_point_geom_EU.sh $windpower_vector $windpower_raster $windpower_EU_raster_scaled

terra 1.7.78
Rasterization complete. Output saved to: /media/sf_Granular/beauty/data/processed/osm/len_powerlines_DE_4647.tif 
terra 1.7.78
Error: [writeRaster] cannot guess file type from filename
Execution halted
0...10...20...30...40...50...60...70...80...90...100 - done.
Rasterization complete. Output saved to: /media/sf_Granular/beauty/data/processed/osm/freq_windpowerplants_DE_4647.tif


# Corine Landcover (CLC)

In [1]:
# import paths from config.py
from rural_beauty.config import CLC_EU, CLC_boolean_layers_dir, CLC_coverage_EU_dir
from rural_beauty import split_CLC_layers

split_CLC_layers.main(CLC_EU, CLC_boolean_layers_dir)

os.listdir(CLC_boolean_layers_dir)


This is the rural_beauty module


['code_acker.tif',
 'code_heide.tif',
 'code_natgru.tif',
 'code_noveg.tif',
 'code_obst.tif',
 'code_seemee.tif',
 'code_spfr.tif',
 'code_stoer.tif',
 'code_wald.tif']

In [2]:
# pass the coverage output folder and the boolean layers folder to the OS for the next step
os.environ['CLC_boolean_layers_dir'] = str(CLC_boolean_layers_dir)
os.environ['CLC_coverage_dir'] = str(CLC_coverage_EU_dir)


In [4]:
!../scripts/resample_CLC_DE.sh $CLC_boolean_layers_dir $CLC_coverage_EU_dir

File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_acker.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_heide.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_natgru.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_noveg.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_obst.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_seemee.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_spfr.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_stoer.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_wald.tif already exists, skipping...
Resampling complete.

# Hemerobieindex


In [3]:
from rural_beauty.config import heme2012_DE, heme2012_DE_repojected

os.environ['heme2012_DE'] = str(heme2012_DE)
os.environ['heme2012_DE_repojected'] = str(heme2012_DE_repojected)

In [6]:
%%bash 
echo "Input: $heme2012_DE"
echo "Output: $heme2012_DE_repojected"

../scripts/reproject_heme.sh $heme2012_DE $heme2012_DE_reprojected

Input: /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif
Output: /media/sf_Granular/beauty/data/cleaned/hemerobie_IOER/heme2012.tif


Creating output file that is 642P x 867L.
Processing /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif [1/1] : 0Using internal nodata values (e.g. -9998) for image /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif.
Copying nodata values from source /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif to destination /media/sf_Granular/beauty/data/cleaned/hemerobie_IOER/heme2012.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


# Protected Areas

In [1]:
from rural_beauty.config import protected0, protected1, protected2, protected_EU, protected_raster

os.environ['protected0'] = str(protected0)
os.environ['protected1'] = str(protected1)
os.environ['protected2'] = str(protected2)

# intermediary stack of EU based polygons of protected areas
os.environ['protected_EU'] = str(protected_EU)

# raster of 'protected area within px'
os.environ['protected_raster'] = str(protected_raster)

This is the rural_beauty module


In [ ]:
%%bash 
echo "Output: $protected_EU"

../scripts/WDPA_subset_reproject.sh $protected0 $protected1 $protected2 $protected_EU

In [ ]:
%%bash
echo "Rasterizing... Output: $protected_raster"

../scripts/rasterize_protected_poly_geom_EU.R $protected_EU $protected_raster